In [1]:
import sys
import numpy as np
import argparse

import scipy.io

import theano
import theano.tensor as T

import lasagne

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled)


In [2]:
from sklearn import preprocessing
from sklearn.externals import joblib

from spacy.en import English

In [3]:
import numpy as np
import pandas as pd

In [4]:
sys.path.append('/home/mayank/visual_question_ans/tools/')

In [5]:
from tools2 import selectFrequentAnswers, grouper
from feature_extraction import get_answers_matrix, get_questions_matrix_sum, get_images_matrix

In [6]:
questions_train = open('../Datasets/VQA/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../Datasets/VQA/preprocessed/answers_train2014_top1000.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../Datasets/VQA/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()
vgg_model_path = '../Datasets/VQA/coco/vgg_feats.mat'
maxAnswers = 1000
questions_train, answers_train, images_train = selectFrequentAnswers(questions_train,answers_train,images_train, maxAnswers)


In [7]:
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(answers_train)

LabelEncoder()

In [8]:
num_classes = len(labelencoder.classes_)
features_ = scipy.io.loadmat(vgg_model_path)
img_ids = open('/home/mayank/visual_question_ans/features/coco_vgg_IDMap.txt', 'r').read().splitlines()

In [9]:
VGG_features = features_['feats']

In [10]:
id_map = {}
for id_pair in img_ids:
    id_pair_split = id_pair.split()
    id_map[id_pair_split[0]] = int(id_pair_split[1])

In [11]:
nlp = English()

In [ ]:
img_dim = 4096
word_vec_dim = 300
nonlin = lasagne.nonlinearities.rectify
depth = 2
num_hidden = 512

network = lasagne.layers.InputLayer(shape=(None,1, img_dim+word_vec_dim))
network = lasagne.layers.dropout(network, p = 0.2)
for _ in range(depth):
    network = lasagne.layers.DenseLayer(network, num_hidden, nonlinearity=nonlin)
    network = lasagne.layers.dropout(network, p = 0.5)
    
#Output layer
softmax = lasagne.nonlinearities.softmax
network = lasagne.layers.DenseLayer(network, num_classes, nonlinearity=softmax)


In [ ]:
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')

prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [ ]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adam(loss, params)

In [ ]:
y = labelencoder.transform(answers_train)
num_classes = labelencoder.classes_.shape[0]


In [ ]:
Y = pd.get_dummies(y)

In [ ]:
del features_

In [ ]:
def exp():
    r = range(1000)
    Y = pd.get_dummies(r)
    return Y

In [ ]:
import pandas as pd

In [ ]:
Y = exp()

In [ ]:
get_questions_matrix_sum(questions_train[0:100], nlp)

In [ ]:
def get_questions_matrix_sum(questions, nlpProcessor):

	num_samples = len(questions)
	
	word_vec_dim = nlpProcessor(questions[0])[0].vector.shape[0]
	questions_matrix = np.zeros((num_samples, word_vec_dim))
	print np.shape(questions_matrix)
	for i in xrange(num_samples):
		tokens = nlpProcessor(questions[i])
		for j in xrange(len(tokens)):
			questions_matrix[i,:] += tokens[j].vector

	return 

In [13]:
X_ques_batch = get_questions_matrix_sum(questions_train[0:100], nlp)
print 'below this'
print np.shape(X_ques_batch)

X_img_batch = get_images_matrix(images_train[0:100], id_map, VGG_features)
print np.shape(X_img_batch)
X_batch = np.hstack((X_ques_batch, X_img_batch))
print np.shape(X_batch)

below this
(100, 300)
(100, 4096)
(100, 4396)


In [17]:
print np.shape(get_answers_matrix(answers_train[0:1000], labelencoder))

(1000, 260)


In [48]:
y = labelencoder.transform(answers_train[0:130])

In [53]:
l = preprocessing.OneHotEncoder()

In [54]:
ll = l.fit_transform(y)

/home/mayank/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [69]:
l = preprocessing.label_binarize(1000,y)


TypeError: Singleton array array(1000) cannot be considered a valid collection.

In [65]:
l.fit_transform(y)

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])